In [4]:
# A Simple RAG Pipeline
import os
from dotenv import load_dotenv, dotenv_values
from utils import clean_text, print_chunks, reset_pinecone, embed_and_upsert

# Load All ENV Values
load_dotenv()
pinecone_namespace="simple-rag-3"
enable_split=False

In [5]:
# 1. Load the txt file
with open('docs/the-judicial-system-of-pakistan.txt', 'r', encoding='utf-8') as file:
    # Read the content of the file
    text = file.read()

print(text)

Established 1988

The Judicial System
of Pakistan

Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad

Established 1988

The Judicial System
of Pakistan

Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad

1st edition

August

2006

2nd edition

May

2007

3rd edition

February 2011

4th edition

May

2015

The Judicial System of Pakistan

Contents
S. No
1
2

3
4

5
6
7
8

9

10

11

12

Introduction
Historical Retrospect
2.1
Hindu Period
2.2
Muslim Period
2.3
British Period
Post-Independence Evolution
Superior Judiciary
4.1
New Procedure for Appointment
4.2
Accountability
4.3
Supreme Court
4.4
High Courts
4.5
Federal Shariat Court
Subordinate Courts
Special Courts and Administrative Tribunals
6.1
Service Tribunals
Procedural Law
Terms and Conditions of Service of Subordinate Judiciary
8.1
Workload
8.2
National Judicial Policy
Organization of Judicial Administration
9.1
Organogram of Courts
9.2
Strength of Judges
9.3
Superior Courts
9.4
Subordina

In [6]:
print("----- Cleaned Text -----")
print(clean_text(text))

----- Cleaned Text -----
Established 1988
The Judicial System
of Pakistan
Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad
Established 1988
The Judicial System
of Pakistan
Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad
1st edition
August
nd edition
May
rd edition
February 2011
4th edition
May
2015
The Judicial System of Pakistan
Contents
S. No

Introduction
Historical Retrospect
2.1
Hindu Period
2.2
Muslim Period
2.3
British Period
Post-Independence Evolution
Superior Judiciary
4.1
New Procedure for Appointment
4.2
Accountability
4.3
Supreme Court
4.4
High Courts
4.5
Federal Shariat Court
Subordinate Courts
Special Courts and Administrative Tribunals
6.1
Service Tribunals
Procedural Law
Terms and Conditions of Service of Subordinate Judiciary
8.1
Workload
8.2
National Judicial Policy
Organization of Judicial Administration
9.1
Organogram of Courts
9.2
Strength of Judges
9.3
Superior Courts
9.4
Subordinate Courts
9.5
Administrative Staff of 

In [8]:
# 2. Create Chunks

from langchain_experimental.text_splitter import SemanticChunker
from langchain_voyageai import VoyageAIEmbeddings

if(enable_split):
    text_splitter = SemanticChunker(VoyageAIEmbeddings(
        voyage_api_key=os.getenv("VOYAGE_API_KEY"), model="voyage-large-2"))
    docs = text_splitter.create_documents([clean_text(text)])
    # Create chunks, extract text
    texts = list(map((lambda x: x.page_content), docs))
    print_chunks(texts)

batch size None
Total Chunks:  31
--------
CHUNK 1
--------
Established 1988
The Judicial System
of Pakistan
Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad
Established 1988
The Judicial System
of Pakistan
Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad
1st edition
August
nd edition
May
rd edition
February 2011
4th edition
May
2015
The Judicial System of Pakistan
Contents
S. No

Introduction
Historical Retrospect
2.1
Hindu Period
2.2
Muslim Period
2.3
British Period
Post-Independence Evolution
Superior Judiciary
4.1
New Procedure for Appointment
4.2
Accountability
4.3
Supreme Court
4.4
High Courts
4.5
Federal Shariat Court
Subordinate Courts
Special Courts and Administrative Tribunals
6.1
Service Tribunals
Procedural Law
Terms and Conditions of Service of Subordinate Judiciary
8.1
Workload
8.2
National Judicial Policy
Organization of Judicial Administration
9.1
Organogram of Courts
9.2
Strength of Judges
9.3
Superior Courts
9.4
Subordinate 

In [9]:
# 3. Reset Pinecone and Upsert Chunks. Only Run Once
#if(enable_split):
    #reset_pinecone(pinecone_namespace)
    #embed_and_upsert(texts, pinecone_namespace)

True

In [19]:
from utils import get_relevant_chunks

# https://chatgpt.com/share/271c8eea-0efc-4c6d-9e76-391a11eecc1b
queries = ["Describe the hierarchy and organizational structure of the judicial system in Pakistan",
           "Explain the reasons behind the discrepancy in the number of judges across different High Courts in Pakistan", 
           "What are the challenges faced by subordinate courts in Pakistan", 
           "Discuss the types of special courts and administrative tribunals in Pakistan and their jurisdictions", 
           "What are the procedural laws governing civil and criminal proceedings in Pakistan?", 
           "How does the appointment and independence of service tribunal members in Pakistan work?", 
           "Analyze the impact of the 2007 Provisional Constitution Order (PCO) on the strength of judges in the Supreme Court and High Courts of Pakistan", 
           "What is the role of the Federal Shariat Court in Pakistan's judicial system?", 
           "Summarize the annual performance of the judiciary in terms of case disposal and pendency as of 2013",
           "What is the significance of the Qanun-e-Shahadat Order 1984 in Pakistan’s legal proceedings?"]

query = queries[2]
relevant_chunks = get_relevant_chunks(query, pinecone_namespace, 10)
relevant_chunks_text = [item["metadata"]["text"] for item in relevant_chunks]

print_chunks(relevant_chunks, True)

# The closest chunk to the queries[0] is on 7th number based on similarity, which is not good

Total Chunks:  10
--------
CHUNK 1
--------
Id: 3, Score: 0.856844723
The Subordinate
Courts function under the administrative control of the High Court. All such courts are funded by the
Provincial Government. 4. Superior Judiciary
The Constitution of Pakistan deals with the superior judiciary in a fairly comprehensive manner and
contains elaborate provisions on the composition, jurisdiction, powers and functions of these
courts. The Constitution provides for the “separation of judiciary from the executive” and the
“independence of judiciary”13 It entrusts the superior courts with an obligation to “preserve, protect
and defend” the Constitution.14 The qualifications of judges, their mode of appointment,15 service
conditions, salary, pension,16 etc. are also laid down in the Constitution. The remuneration of judges
and other administrative expenditures of the superior courts are charged on the Federal/Provincial
Consolidated Fund,17 which means it may be discussed but cannot be voted u

In [20]:
from utils import rerank_chunks

# Lets use a reranker to see how much it helps
rerank_results = rerank_chunks(query, relevant_chunks_text)
print_chunks(rerank_results, True)

Total Chunks:  10
--------
CHUNK 1
--------
Id: 2, Score: 0.2451171875
The strength of judges in the

Judicial Statistics of Pakistan, op.cit. Judicial Statistics of Pakistan, op.cit. 19
The Judicial System of Pakistan
Supreme Court came down from 29 to 18, following a ruling of this Court in the case of Sindh High
Court Bar Association v Federation wherein the judges who had taken oath under the Provisional
Constitution Order (PCO) 2007. Similarly, close to one hundred judges of High Courts were also hit by
the ruling and were denotified. The High Courts then took some time to regain their strength,
through fresh induction. The Subordinate Courts continue to face the daunting challenge of shortage of strength, support
staff, equipment and other budgetary constraints. The judiciary – 3rd pillar of the State – is not
getting even 1% of the total budgetary allocation of the federal/provincial governments. Notwithstanding the deficiency in strength and budgetary constraints, due to effic

In [21]:
# Only the Top Chunks should be given to llm, so we choose chunks with a score of 0.8
top_chunks = [item for item in rerank_results if item["score"] >= 0.6]
print_chunks(top_chunks, True)

Total Chunks:  0
